# EMSN 2.0 - Vocalization Classifier Training (Colab GPU)

Train alle 232 Nederlandse vogelsoorten met GPU acceleratie.

**Geschatte tijd:** 1-2 dagen (vs 9 maanden op NAS CPU)

## Setup
1. Upload je spectrogrammen naar Google Drive
2. Vul je database credentials in
3. Run alle cellen

In [ ]:
# Check GPU availability
!nvidia-smi
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Install dependencies
!pip install psycopg2-binary librosa scikit-learn matplotlib seaborn tqdm -q

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Maak werkdirectories
import os
DRIVE_BASE = '/content/drive/MyDrive/EMSN-Vocalization'
os.makedirs(f'{DRIVE_BASE}/models', exist_ok=True)
os.makedirs(f'{DRIVE_BASE}/logs', exist_ok=True)
print(f"Werkdirectory: {DRIVE_BASE}")

In [ ]:
# === CONFIGURATIE ===
# Vul hier je database credentials in
# (of upload een .env bestand naar Drive)

DB_CONFIG = {
    'host': 'JOUW_PUBLIEKE_IP',  # Je router's publieke IP of een tunnel
    'port': 5433,
    'database': 'emsn',
    'user': 'birdpi_zolder',
    'password': 'JOUW_WACHTWOORD'
}

# Training parameters
EPOCHS = 25
BATCH_SIZE = 32  # Groter mogelijk met GPU
LEARNING_RATE = 0.001
VERSION = '2025'  # Jaarversie i.p.v. kwartaal

In [ ]:
# Database connectie test
import psycopg2

def get_db():
    return psycopg2.connect(**DB_CONFIG)

try:
    conn = get_db()
    cur = conn.cursor()
    cur.execute("SELECT COUNT(*) FROM vocalization_training")
    count = cur.fetchone()[0]
    print(f"✅ Database verbonden! {count} soorten in training tabel.")
    cur.close()
    conn.close()
except Exception as e:
    print(f"❌ Database error: {e}")
    print("\nTips:")
    print("1. Zet port forwarding aan op je router (5433 -> NAS)")
    print("2. Of gebruik ngrok/cloudflare tunnel")
    print("3. Check of PostgreSQL externe connecties accepteert")

In [ ]:
# CNN Model (zelfde architectuur als NAS versie)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class VocalizationCNN(nn.Module):
    """CNN voor vocalisatie classificatie (song/call/alarm)."""
    
    def __init__(self, input_shape=(128, 128), num_classes=3):
        super().__init__()
        
        self.features = nn.Sequential(
            # Conv block 1
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25),
            
            # Conv block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25),
            
            # Conv block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.25),
        )
        
        # Bereken flatten size
        h, w = input_shape[0] // 8, input_shape[1] // 8
        flatten_size = 128 * h * w
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flatten_size, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Test model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VocalizationCNN().to(device)
print(f"Model op: {device}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Spectrogrammen laden vanuit Drive
# Je moet eerst je spectrogrammen uploaden naar Drive!

import numpy as np
from pathlib import Path
import json

SPECTROGRAMS_DIR = Path(f'{DRIVE_BASE}/spectrograms')

def load_spectrograms_for_species(species_dir):
    """Laad spectrogrammen voor een soort."""
    X, y = [], []
    label_map = {'song': 0, 'call': 1, 'alarm': 2}
    
    for label_name, label_idx in label_map.items():
        label_dir = species_dir / label_name
        if not label_dir.exists():
            continue
        
        for npy_file in label_dir.glob('*.npy'):
            try:
                spec = np.load(npy_file)
                # Resize naar 128x128 als nodig
                if spec.shape != (128, 128):
                    from skimage.transform import resize
                    spec = resize(spec, (128, 128), anti_aliasing=True)
                X.append(spec)
                y.append(label_idx)
            except Exception as e:
                print(f"  Skip {npy_file}: {e}")
    
    return np.array(X), np.array(y)

# Check welke soorten beschikbaar zijn
if SPECTROGRAMS_DIR.exists():
    species_dirs = [d for d in SPECTROGRAMS_DIR.iterdir() if d.is_dir()]
    print(f"Gevonden: {len(species_dirs)} soorten in Drive")
else:
    print(f"❌ Directory niet gevonden: {SPECTROGRAMS_DIR}")
    print("\nUpload je spectrogrammen naar Google Drive:")
    print(f"  {DRIVE_BASE}/spectrograms/<soort>/song/*.npy")
    print(f"  {DRIVE_BASE}/spectrograms/<soort>/call/*.npy")
    print(f"  {DRIVE_BASE}/spectrograms/<soort>/alarm/*.npy")

In [ ]:
# Training functie
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import time

def train_species(species_name, species_dir, version=VERSION):
    """Train model voor één soort."""
    print(f"\n{'='*50}")
    print(f"Training: {species_name}")
    print(f"{'='*50}")
    
    start_time = time.time()
    
    # Laad data
    X, y = load_spectrograms_for_species(species_dir)
    
    if len(X) < 100:
        print(f"  ⚠️ Te weinig data: {len(X)} spectrogrammen (min 100)")
        return None, 'insufficient_data'
    
    print(f"  Data: {len(X)} spectrogrammen")
    
    # Check class balance
    unique, counts = np.unique(y, return_counts=True)
    if len(unique) < 2:
        print(f"  ⚠️ Maar {len(unique)} klasse(s), minimaal 2 nodig")
        return None, 'insufficient_classes'
    
    print(f"  Klassen: {dict(zip(['song', 'call', 'alarm'], counts))}")
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Naar tensors
    X_train = torch.FloatTensor(X_train).unsqueeze(1)  # Add channel dim
    X_val = torch.FloatTensor(X_val).unsqueeze(1)
    y_train = torch.LongTensor(y_train)
    y_val = torch.LongTensor(y_val)
    
    # DataLoaders
    train_loader = DataLoader(
        TensorDataset(X_train, y_train),
        batch_size=BATCH_SIZE,
        shuffle=True
    )
    val_loader = DataLoader(
        TensorDataset(X_val, y_val),
        batch_size=BATCH_SIZE
    )
    
    # Model
    model = VocalizationCNN(num_classes=len(unique)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Training loop
    best_acc = 0
    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}
    
    for epoch in range(EPOCHS):
        # Train
        model.train()
        train_loss, train_correct = 0, 0
        
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_correct += (outputs.argmax(1) == y_batch).sum().item()
        
        # Validate
        model.eval()
        val_loss, val_correct = 0, 0
        
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                val_loss += criterion(outputs, y_batch).item()
                val_correct += (outputs.argmax(1) == y_batch).sum().item()
        
        # Metrics
        train_acc = train_correct / len(X_train)
        val_acc = val_correct / len(X_val)
        
        history['loss'].append(train_loss / len(train_loader))
        history['val_loss'].append(val_loss / len(val_loader))
        history['accuracy'].append(train_acc)
        history['val_accuracy'].append(val_acc)
        
        if val_acc > best_acc:
            best_acc = val_acc
        
        if (epoch + 1) % 5 == 0:
            print(f"  Epoch {epoch+1}/{EPOCHS} - val_acc: {val_acc:.1%}")
    
    # Save model
    dirname = species_dir.name
    model_filename = f"{dirname}_cnn_{version}.pt"
    model_path = Path(f'{DRIVE_BASE}/models/{model_filename}')
    
    torch.save({
        'model_state_dict': model.state_dict(),
        'num_classes': len(unique),
        'accuracy': best_acc,
        'history': history,
        'species_name': species_name,
        'version': version
    }, model_path)
    
    elapsed = time.time() - start_time
    print(f"  ✅ Klaar! Accuracy: {best_acc:.1%}, Tijd: {elapsed/60:.1f} min")
    print(f"  Model: {model_path}")
    
    return best_acc, 'success'

In [ ]:
# Train alle soorten
from datetime import datetime

results = []
start_all = time.time()

print(f"Start training: {datetime.now()}")
print(f"Soorten: {len(species_dirs)}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print()

for i, species_dir in enumerate(sorted(species_dirs)):
    species_name = species_dir.name.replace('_', ' ').title()
    print(f"[{i+1}/{len(species_dirs)}] ", end='')
    
    try:
        acc, status = train_species(species_name, species_dir)
        results.append({
            'species': species_name,
            'accuracy': acc,
            'status': status
        })
    except Exception as e:
        print(f"  ❌ Error: {e}")
        results.append({
            'species': species_name,
            'accuracy': None,
            'status': f'error: {str(e)[:50]}'
        })

elapsed_all = time.time() - start_all
print(f"\n{'='*50}")
print(f"Totale tijd: {elapsed_all/3600:.1f} uur")
print(f"Succesvol: {sum(1 for r in results if r['status'] == 'success')}")
print(f"Mislukt: {sum(1 for r in results if r['status'] != 'success')}")

In [ ]:
# Resultaten opslaan
import json
import pandas as pd

# Als DataFrame
df = pd.DataFrame(results)
df.to_csv(f'{DRIVE_BASE}/training_results_{VERSION}.csv', index=False)

# Samenvatting
successful = df[df['status'] == 'success']
print(f"\nSamenvatting:")
print(f"  Getraind: {len(successful)} soorten")
print(f"  Gem. accuracy: {successful['accuracy'].mean():.1%}")
print(f"  Min accuracy: {successful['accuracy'].min():.1%}")
print(f"  Max accuracy: {successful['accuracy'].max():.1%}")

# Top 10
print(f"\nTop 10 beste modellen:")
print(successful.nlargest(10, 'accuracy')[['species', 'accuracy']].to_string(index=False))

In [ ]:
# Update database met resultaten (optioneel)
# Dit werkt alleen als je database extern bereikbaar is

def update_database_with_results(results):
    """Update vocalization_model_versions tabel."""
    try:
        conn = get_db()
        cur = conn.cursor()
        
        for r in results:
            if r['status'] != 'success':
                continue
            
            dirname = r['species'].lower().replace(' ', '_')
            model_file = f"{dirname}_cnn_{VERSION}.pt"
            
            cur.execute("""
                INSERT INTO vocalization_model_versions 
                (species_name, version, model_file, accuracy, is_active, notes)
                VALUES (%s, %s, %s, %s, TRUE, 'Trained on Colab GPU')
                ON CONFLICT (species_name, version) 
                DO UPDATE SET accuracy = EXCLUDED.accuracy, 
                              model_file = EXCLUDED.model_file,
                              trained_at = NOW()
            """, (r['species'], VERSION, model_file, r['accuracy']))
        
        conn.commit()
        print(f"✅ Database bijgewerkt met {len([r for r in results if r['status'] == 'success'])} modellen")
        cur.close()
        conn.close()
    except Exception as e:
        print(f"❌ Database update mislukt: {e}")
        print("Je kunt de modellen handmatig kopiëren naar de NAS")

# Uncomment om database te updaten:
# update_database_with_results(results)

## Na de training

De getrainde modellen staan nu in Google Drive:
- `EMSN-Vocalization/models/*.pt`

Om ze te gebruiken op je NAS:

1. Download de `models/` map van Drive
2. Kopieer naar `/volume1/docker/emsn-vocalization/data/models/`
3. De modellen worden automatisch herkend